<a href="https://colab.research.google.com/github/JuIsa/Dominant-Color/blob/main/dominant_color.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import scipy.cluster
import sklearn.cluster
import numpy as np
from PIL import Image
import os
import cv2
import moviepy.editor
import warnings
warnings.filterwarnings("ignore")


In [3]:
def get_dominant_colors(image):  # 
    scale = 3
    image = image.crop((0, 90, image.width, 628))
    image = image.resize((image.width//scale, image.height//scale))# optional, to reduce time
    ar = np.asarray(image)
    shape = ar.shape
    ar = ar.reshape(np.product(shape[:2]), shape[2]).astype(float)

    kmeans = sklearn.cluster.MiniBatchKMeans(
        n_clusters=10,
        init="k-means++",
        max_iter=20,
        random_state=1000
    ).fit(ar)
    codes = kmeans.cluster_centers_

    vecs, _dist = scipy.cluster.vq.vq(ar, codes)         # assign codes
    counts, _bins = np.histogram(vecs, len(codes))    # count occurrences

    colors = []
    for index in np.argsort(counts)[::-1]:
        colors.append(tuple([int(code) for code in codes[index]]))
    return colors         

In [5]:
def get_frames(vid, output, rate=0.33, frameName='frame'):
    vidcap = cv2.VideoCapture(vid)
    clip = moviepy.editor.VideoFileClip(vid)
    seconds = clip.duration
    print('durration: ' + str(seconds))
    
    count = 0
    frame = 0
    
    if not os.path.isdir(output):
        os.mkdir(output)
    
    success = True
    while success:
        vidcap.set(cv2.CAP_PROP_POS_MSEC,frame*1000)      
        success,image = vidcap.read()

        ## Stop when last frame is identified
        print(frame)
        if frame > seconds or not success:
            break

        print('extracting frame ' + frameName + '-%d.png' % count)
        name = output + '/' + frameName + '-%d.png' % count # save frame as PNG file
        cv2.imwrite(name, image)
        frame += rate
        count += 1

In [ ]:
def generate_image(n):
  img_sp = np.zeros((30, n, 3), dtype=np.uint8)
  for i in range(n):
    img = Image.open('spider/frame-'+str(i)+'.png')
    colors = get_dominant_colors(img)
    img_sp[:,i] = colors[0]
    if i%25==0:
      print(i,' is done ',n)
  print('Done')
  return img_sp

In [8]:
def save_img(img_sp):
  res = Image.fromarray(img_sp, "RGB")
  res.show()
  image_filename = "color-line.png"
  res.save(image_filename)

In [ ]:
def delete_images():
  import os
  for i in range(n):
    os.remove('spider/frame-'+str(i)+'.png')

In [ ]:
n = get_frames('drive/MyDrive/spiderverse.mp4', 'spider')
img = generate_image(n)
save_img(img)